## 1. Linear regression diagnostics and modeling using StatsModels library: 
### StatsModels 라이브러리를 사용한 선형 회귀 진단 및 모델링 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf  #api가져와 smf라고 이름 지음
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn import metrics
%matplotlib inline

### 1.1. Load the 'Boston' dataset from Scikit-Learn and convert it into a DataFrame: 
#### Scikit-Learn에서 'Boston'데이터 세트를 로드하고 DataFrame으로 변환

In [2]:
data = load_boston()

In [3]:
# The explanatory variables.
X = data['data']
header = data['feature_names']

In [4]:
# The response variable.
Y = data['target']
Y = Y.reshape(-1, 1) #행과열 갯수맞추기 위함

In [5]:
# Transform into a DataFrame.
df = pd.DataFrame(np.append(X,Y,axis = 1))
df.columns = list(header) + ['PRICE']

In [6]:
df.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### 1.2. Linear modeling using R-styled formula: R 공식을 사용한 선형 모델링 

In [7]:
#모든 설명변수 사용
myModel1 = smf.ols(formula = "PRICE ~ CRIM + ZN + INDUS + CHAS + NOX + RM + AGE + DIS + RAD + TAX + PTRATIO + B + LSTAT ", data=df)
#반응변수 price, 설명변수 crim~ (이 부분은 칼럼이름하고 일치해야 함)

result1 = myModel1.fit() #학습
result1.summary()        #요약

#결정계수 0.741 > 조정결정계수 0.734

#F-statistic: F검정 해석 필요없음   
#Prob (F-statistic): F검정의 p값이 6.72e로 매우작다 => 귀무가설 기각, 대립가설 채택 (최소 설명력을 제공한다는 것이 검증됨)

#AIC 3026

#INDUS와 AGE는 p값이 0.05보다 커서 설명력이 없음

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  PRICE   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Tue, 21 Jan 2020   Prob (F-statistic):          6.72e-135
Time:                        11:22:17   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     36.4595      5.103      7.144      0.000      26.432      46.487
CRIM          -0.1080      0.033     -3.287      0.001      -0.173      -0.043
ZN             0.0464      0.014      3.382      0.001       0.019       0.073
INDUS          0.0206      0.061      0.334      0.738      -0.100       0.141
CHAS           2.6867      0.862      3.118      0.002       0.994       4.380
NOX          -17.7666      3.820     -4.651      0.000     -25.272     -10.262
RM             3.8099      0.418      9.116      0.000       2.989       4.631
AGE            0.0007      0.013      0.052      0.958      -0.025       0.027
DIS           -1.4756      0.199     -7.398      0.000      -1.867      -1.084
RAD            0.3060      0.066      4.613      0.000       0.176       0.436
TAX           -0.0123      0.004     -3.280      0.001      -0.020      -0.005
PTRATIO       -0.9527      0.131     -7.283      0.000      -1.210      -0.696
B              0.0093      0.003      3.467      0.001       0.004       0.015
LSTAT         -0.5248      0.051    -10.347      0.000      -0.624      -0.425
==============================================================================
Omnibus:                      178.041   Durbin-Watson:                   1.078
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              783.126
Skew:                           1.521   Prob(JB):                    8.84e-171
Kurtosis:                       8.281   Cond. No.                     1.51e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.51e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

**NOTE: INDUS와 AGE는 p value가 0.05보다 커서 설명력이 없음: 통계적 유의성이 없다.  Adjusted $R^2 = 0.734$ and $AIC = 3026$.** 

### 1.3 불필요한 데이터 제거 후 다시 모델링:

In [8]:
#INDUS와 AGE 제거 후 다시 분석
myModel2 = smf.ols(formula = "PRICE ~ CRIM + ZN + CHAS + NOX + RM + DIS + RAD + TAX + PTRATIO + B + LSTAT ", data=df)
result2 = myModel2.fit()
result2.summary()

#이제 모든 p값이 0.05보다 작음: 설명력을 제공한다

#결정계수는 같고, 조정결정계수는 조금씩 향상

#AIC도 조금 향상됨(낮아질수록 좋은 수치)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  PRICE   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.735
Method:                 Least Squares   F-statistic:                     128.2
Date:                Tue, 21 Jan 2020   Prob (F-statistic):          5.54e-137
Time:                        11:22:17   Log-Likelihood:                -1498.9
No. Observations:                 506   AIC:                             3022.
Df Residuals:                     494   BIC:                             3072.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     36.3411      5.067      7.171      0.000      26.385      46.298
CRIM          -0.1084      0.033     -3.307      0.001      -0.173      -0.044
ZN             0.0458      0.014      3.390      0.001       0.019       0.072
CHAS           2.7187      0.854      3.183      0.002       1.040       4.397
NOX          -17.3760      3.535     -4.915      0.000     -24.322     -10.430
RM             3.8016      0.406      9.356      0.000       3.003       4.600
DIS           -1.4927      0.186     -8.037      0.000      -1.858      -1.128
RAD            0.2996      0.063      4.726      0.000       0.175       0.424
TAX           -0.0118      0.003     -3.493      0.001      -0.018      -0.005
PTRATIO       -0.9465      0.129     -7.334      0.000      -1.200      -0.693
B              0.0093      0.003      3.475      0.001       0.004       0.015
LSTAT         -0.5226      0.047    -11.019      0.000      -0.616      -0.429
==============================================================================
Omnibus:                      178.430   Durbin-Watson:                   1.078
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              787.785
Skew:                           1.523   Prob(JB):                    8.60e-172
Kurtosis:                       8.300   Cond. No.                     1.47e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.47e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

**NOTE: 이제는 모든 선형변수들이 통계적으로 유의하다. Adjusted $R^2 = 0.735$ and $AIC = 3022$.** 

**CONCLUSION: 두 번째 모델은 첫 번째 모델에 비해 약간 최적화되었다.**